In [1]:
#export
import gc
from functools import partial
from torch import nn
import torch
from torchvision.models import vgg19_bn
vgg = vgg19_bn(True).features.cuda().eval()
blocks = [i for i,m in enumerate(vgg) if isinstance(m, nn.ReLU)]
blocks = [blocks[i] for i in range(0,len(blocks),2)]
blocks = blocks[:3]

class Hook():
    def __init__(self,m,f): self.hook = m.register_forward_hook(partial(f,self))
    def remove(self):
        self.hook.remove()
        del self.o
        gc.collect()
    def __del__(self): del self.hook,self.o

def hook_ouptut(hook,mod,inp,out):
    if not hasattr(hook,'o'): hook.o = None
    hook.o = out.detach()

class Feature_loss():
    def __init__(self,model,blocks,func):
        self.m = model
        self.b = blocks
        self.f = func
        self.hooks = [Hook(self.m[i],self.f) for i in self.b]

    def __call__(self,pred,targ):
        pred_features = self.make_features(pred)
        targ_features = self.make_features(targ)
        self._remove()
        loss_list = [self.one_layer_loss(pred_features[i], targ_features[i]) for i in range(len(pred_features))]
        return sum(loss_list)


    def make_features(self,x):
        self.m(x)
        l = [i.o for i in self.hooks]
        return l

    def one_layer_loss(self,pf,tf):
#         pf,tf = pf.cuda(),tf.cuda()
        a = pf[0].numel()
        err = ((pf - tf)**2).sum(dim = [1,2,3])
        return err/(2*a)

    def _remove(self):
        for i in self.hooks: del i.o

    def __del__(self):
        for i in self.hooks:
            i.remove()

class Total_loss():
    def __init__(self,blocks,alpha = 1,beta = 0.5,r = 10000, attr = 'enc',perceptual_model = vgg,func = hook_ouptut):
        self.r = r
        self.a = attr
        self.kloss = 0.
        self.ploss_func = Feature_loss(perceptual_model,blocks,func)
        self.al,self.be = alpha,beta

    def __call__(self,model,pred,targ):
        self.kloss = self.kl_loss(model,self.a)
        self.ploss = self.ploss_func(pred,targ).sum()
        return self.al*self.kloss + self.be*self.ploss

    def kl_loss(self,m,attr):
        e = getattr(m,attr)
        return (-0.5*torch.sum(1 + e.log_var - torch.pow(e.mean,2) - torch.exp(e.log_var), axis = 1)).mean()

In [3]:
!python notebook2script.py Loss.ipynb


Converted Loss.ipynb to exp\nb_Loss.py
